In [16]:
#    This is a test
corpus = 'data/delorme.com_shu.pages_89.txt'


import math
import os
import sys

from tests import tests1, tests2

from normalizer import Normalizer
from lib.CharacterIndex import CharacterIndex
from lib.NaiveTokenizer import NaiveTokenizer
from lib.TextStreamer import TextStreamer

from lib.Tools import (
    FreqDist
)

from collections import (
    Counter,
    defaultdict as deft
)



def freq_ratio(maxim, minim, log_base=2.0):
    if maxim < 10:
        return False
    multiplier = math.log(maxim, log_base)
#     print maxim, minim, maxim / minim, multiplier, (multiplier * multiplier)
    if maxim / minim >= (multiplier * multiplier):
        return True
    else:
        return False


def get_name(template):
    i = 1
    while True:
        name = template % i
        if not os.path.exists(name):
            return name
        i += 1

    
CONFIG = {
    'chgram': 2,
    'sim_thres': 0.9,
    'freq_ratio': 10,
    'min_Count': 10
}

tests = tests1.items() + tests2.items()

#	Collect input from large text file:
streamer = TextStreamer(corpus)

#	Preprocessing step
tokenized_stream = NaiveTokenizer(
    streamer,
    to_lower=True
)

#	Map all character n-grams to words, and all words to their
#	character n-grams
index = CharacterIndex(list(tokenized_stream) + tests, min_r=CONFIG['sim_thres'])
index.build()


dump = []
for tokens in list(tokenized_stream) + tests:
    dump += tokens
freq_dist = Counter(dump)

hits = 0
empty = 0


template = 'test.%d.txt'
file = open(get_name(template), 'wb')
file.write('\n'.join(['%s=%s' % (str(x), str(y)) for x, y in CONFIG.items()]) + '\n')
#file.write('CONFIG.items() \n')

for correct, error in tests:
    similars = index[error]
    if not similars:
        empty += 1
    f_w = freq_dist[error]
    if not similars:
        continue
    if similars[0][0] != correct:
#        hits += 1
#        print error, '>', correct, hits, hits / float(len(tests) - empty)
#    else:
        file.write('error="%s"  hypothesis="%s"  human="%s"\n' % (error, similars[0][0], correct))

file.close()

<iter=0 records=2000>
<iter=0 records=4000>
<iter=0 records=6000>
<iter=0 records=8000>
<iter=0 records=10000>
<iter=0 records=12000>
<iter=0 records=14000>
<iter=0 records=16000>
<iter=0 records=18000>
<iter=0 records=20000>
<iter=0 records=22000>
<iter=0 records=24000>
